In [ ]:
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('incidents.csv')


def cast_date_str_to_date_and_sort(dataframe):
    dataframe['DATE'] = pd.to_datetime(dataframe['DATE'])
    dataframe.sort_values(by='DATE', inplace=True)
    new_date_range_filled = pd.date_range(start='2021-01-01', end='2023-01-01', freq='MS')
    dataframe.set_index("DATE", inplace=True)
    new_index = pd.Index(new_date_range_filled, name="DATE")
    dataframe2 = dataframe.reindex(new_index, fill_value=0)
    dataframe2.reset_index(inplace=True)
    dataframe2['DATE'] = pd.to_datetime(dataframe2['DATE']).dt.strftime('%Y-%m')
    
    return dataframe2
    
    
def get_num_of_incidents_for_month_dataframe():
    date_with_num_of_incidents = df.groupby('DATE').size().reset_index(name='num_of_incidents')
    date_with_num_of_incidents = cast_date_str_to_date_and_sort(date_with_num_of_incidents)
    print(date_with_num_of_incidents)
    
    return date_with_num_of_incidents


def draw_line_chart_for_num_of_incidents_with_months(month_and_num_of_incidents, num_of_inc, name_prefix=""):
    x = month_and_num_of_incidents['DATE']
    y = month_and_num_of_incidents[num_of_inc]

    plt.plot(x, y)
    plt.title(f'{name_prefix}Number of incidents from 2021 to 2023')
    plt.xlabel('Month')
    plt.ylabel('Number of incidents')
    plt.xticks(rotation=90)
    plt.show()

    
def get_num_of_incidents_for_month_for_each_country():
    gb_countries = df.groupby('COUNTRY')
    dataframes = []
    for country, datafr in gb_countries:
        date_and_num_of_incidents = datafr.groupby('DATE').size().reset_index(name=country)
        date_and_num_of_incidents = cast_date_str_to_date_and_sort(date_and_num_of_incidents)
        dataframes.append(date_and_num_of_incidents)
    
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['DATE'], how='outer'), dataframes).fillna('0')
    print(df_merged)
    return df_merged
    
    
def draw_stacked_bar(dataframe):
    dates = dataframe['DATE']
    country_names = list(dataframe.columns.values)
    country_names.remove('DATE')

    countries = [dataframe[country_name] for country_name in country_names]
    bottom = 0
    for i, country in enumerate(countries):
        plt.bar(dates, country, bottom=bottom, label=country_names[i])
        bottom+=country
        
    plt.xticks(rotation=90)
    plt.xlabel('Months')
    plt.ylabel('Number of incidents')
    plt.show()


def draw_charts_for_each_country_with_num_of_incidents():
    gb_countries = df.groupby('COUNTRY')
    for country, datafr in gb_countries:
        date_and_num_of_incidents = datafr.groupby('DATE').size().reset_index(name=f"{country}_num_of_incidents")
        date_and_num_of_incidents = cast_date_str_to_date_and_sort(date_and_num_of_incidents)
        draw_line_chart_for_num_of_incidents_with_months(date_and_num_of_incidents, f"{country}_num_of_incidents", f"{country}: ")

  
def draw_multiple_line_charts_in_1_figure(dataframe, column_to_gb):
    gb_countries = df.groupby(column_to_gb)
    num_of_rows = len(gb_countries)//2+1
    fig, axs = plt.subplots(num_of_rows, 2)
    fig.suptitle('Number of incidents amongst countries')
    
    if len(gb_countries)%2:
        axs[num_of_rows-1, 1].remove()
    
    for i, (country, datafr) in enumerate(gb_countries):
        row_num = int(i/2)
        side = i%2
        date_and_num_of_incidents = datafr.groupby('DATE').size().reset_index(name='num_of_incidents')
        date_and_num_of_incidents = cast_date_str_to_date_and_sort(date_and_num_of_incidents)
        x = date_and_num_of_incidents['DATE']
        y = date_and_num_of_incidents['num_of_incidents']
        current_ax = axs[row_num][side]
        current_ax.plot(x, y)
        current_ax.set_ylabel('Incidents', fontsize = 8.0) # Y label
        current_ax.set_xlabel('Months', fontsize = 8.0) # Y label
        current_ax.tick_params(labelrotation=90)
        current_ax.title.set_text(f"{country}: Number of incidents")
    
    plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=1.5,
                    top=0.9,
                    wspace=0.5,
                    hspace=1.8)
    plt.show()

    
#task 1   
number_of_incidents_for_month = get_num_of_incidents_for_month_dataframe()
draw_line_chart_for_num_of_incidents_with_months(number_of_incidents_for_month, "num_of_incidents")

#task 2
dataframe = get_num_of_incidents_for_month_for_each_country()
draw_stacked_bar(dataframe)

#task 3
draw_charts_for_each_country_with_num_of_incidents()

# task 4
draw_multiple_line_charts_in_1_figure(df, 'COUNTRY')




